In [ ]:
!pip install datefinder

In [ ]:

import datefinder
import datetime
import re
import nltk
from nltk import word_tokenize as tokenize
from nltk import pos_tag as tag
nltk.download('wordnet')

# import sutime
# from sutime import SUTime
import json


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [ ]:

#https://www.techwithtim.net/tutorials/voice-assistant/speaking-events/
#https://pypi.org/project/datefinder/

sorted_months = ["january", "february", "march", "april", "may", "june","july",
          "august", "september","october","november", "december"]
sorted_days = ["monday", "tuesday", "wednesday", "thursday", "friday",
        "saturday", "sunday"]
ordinal_numbers = ["rd", "th", "st", "nd"]
numbers_list = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

def date_from_text(input_string):
    '''
    get datetime() date information from raw text
    '''
    
    text = input_string.lower()
    today = datetime.date.today()

    # If today is in text:
    if text.count("today") > 0:
        return today

    #if tomorrow is in text:
    if text.count("tomorrow") > 0 or text.count("tmr") > 0:
        return today + datetime.timedelta(1)

    #For the case where we have in _ days/weeks
    if bool(re.search(r'in\s(\d+|one|two|three|four|five|six|seven|eight|nine)\s(days|weeks|week)', text)):
      x = re.search(r'in\s(\d+|one|two|three|four|five|six|seven|eight|nine)\s(days|weeks|week)', text)
      match = x.group()
      n = match.split()[-2]
      if n in numbers_list:
        num = numbers_list.index(n)+1
      else:
        num = int(n)

      if match.split()[-1] == 'days':
        return today + datetime.timedelta(num)
      else:
        return today + datetime.timedelta(num*7) 

    

    day, month, day_idx = -1,-1,-1
    year = today.year

    for token in text.split():
      #regualar expression for abreviated months:
      if bool(re.search(r'jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?', token)):
        match = re.findall(r'jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?', token)[0]
        #print(match)

        #find corresponding month index
        for m in sorted_months:
          if match in m:
            month = sorted_months.index(m) + 1
            #print(month)
        
      elif bool(re.search(r'\b((mon|tues|wed(nes)?|thur(s)?|fri|sat(ur)?|sun)(day)?)\b', token)):

        match = re.findall(r'\b((mon|tues|wed(nes)?|thur(s)?|fri|sat(ur)?|sun)(day)?)\b', token)[0][0]
        
        #find corresponding day index
        for d in sorted_days:
          if match in d:
            day_idx = sorted_days.index(d)
      
      elif token in sorted_days:
        day_idx = sorted_days.index(token)

      elif token.isdigit():
        day = int(token)
        
      else:
        for ordinal in ordinal_numbers:
          found = token.find(ordinal)
          if found > 0:
            try:
              day = int(token[:found])
            except:
              pass

    #If the given month has passed go to the next year
    if month < today.month and month != -1:  
      year = year+1
    
    #If we are given the current month but day has passed
    #go to the next year 
    if month == today.month and day !=-1:
      if day < today.day:
        year = year+1

    #if month is not present:
    if month == -1 and day != -1:  
      #if day is less than current day go to next month
      if day < today.day:
        month = today.month + 1
      
      else:
        month = today.month

    
    if month == -1 and day == -1 and day_idx != -1:
      today_idx = today.weekday()
      difference = day_idx - today_idx

      if difference < 0:
        difference += 7
        if text.count("next") >= 1:
          difference += 7

      return today + datetime.timedelta(difference)

    if day != -1:  
      return datetime.datetime(month=month, day=day, year=year)
    
    else:
      matches = datefinder.find_dates(input_string)
      for match in matches:
        return match.date()

In [ ]:
def time_from_text(input_string):
  '''
  get time information from raw text
  '''
  
  if bool(re.search(r'(([0-1]{0,1}[0-9]( )?(AM|am|aM|Am|PM|pm|pM|Pm))|(([0]?[1-9]|1[0-2])(:|\.)[0-5][0-9]( )?(AM|am|aM|Am|PM|pm|pM|Pm))|(([0]?[0-9]|1[0-9]|2[0-3])(:|\.)[0-5][0-9]))', input_string)):
    match = re.findall(r'(([0-1]{0,1}[0-9]( )?(AM|am|aM|Am|PM|pm|pM|Pm))|(([0]?[1-9]|1[0-2])(:|\.)[0-5][0-9]( )?(AM|am|aM|Am|PM|pm|pM|Pm))|(([0]?[0-9]|1[0-9]|2[0-3])(:|\.)[0-5][0-9]))', input_string)[0][0]
    match = match.lower()

    match_list = match.split()
    #print(match_list)
    
    if len(match_list) == 1:
      
      if 'pm' in match:
        
        i = match.index('pm')
        num = match[:i]
        
        if ':' in num:
          split = num.split(':')
          hours = int(split[0]) + 12
          minutes = int(split[1])
      
        else:
          hours = int(num) + 12
          minutes = 0
      
      elif 'am' in match:
        i = match.index('am')
        num = match[:i]
        
        if ':' in num:
          split = num.split(':')
          hours = int(split[0])
          minutes = int(split[1])
        else:
          hours = int(num)
          minutes = 0
      
      else:
        split = match.split(':')
        hours = int(split[0])
        minutes = int(split[1])
      
    elif len(match_list) == 2:
      
      if ':' in match_list[0]:
        
        split = match_list[0].split(':')
        hours = int(split[0])
        minutes = int(split[1])
     
      else:
        hours = int(match_list[0])
        minutes = 0
      
      if match_list[-1] == 'pm':
        hours += 12

        
  else:
    
    print('No time in string')
    return None
  #print('Hours:',hours)
  #print('Minutes:', minutes)
  timeobj = datetime.time(hours, minutes)
  return timeobj



In [ ]:
string = 'Next Saturday at 23:00'

# x = re.search(r'in\s\d+\s(days|weeks|week)', string)
# x.group()
date = date_from_text(string)
time = time_from_text(string)
print('String:')
print(string)
print('Date:', date)
print('Time:', time)

String:
Next Saturday at 23:00
Date: 2021-08-21
Time: 23:00:00


In [ ]:
def get_slot_from_datetime(time, date):

  '''
  accepts time in 24h format and date
  returns timeslot
  '''
  hour_lst = str(time).split(':')
  day_slot = int(hour_lst[0])
  #isoweekday() -> Monday = 1 ...  Sunday  = 6
  day_idx = date.isoweekday() % 7 
  timeslot = day_slot+24*day_idx
  return timeslot



In [ ]:
timeslot = get_slot_from_datetime(time, date)

In [ ]:
print(timeslot)

167


In [ ]:
text = 'in 2 days'
if bool(re.search(r'in\s\d+\s(days|weeks|week)', text)):
  print(True)
else:
  print(False)

True


In [ ]:
text = 'in nine days'
x = re.search(r'in\s(\d+|one|two|three|four|five|six|seven|eight|nine)\s(days|weeks|week)', text)
match = x.group()
match.split()


['in', 'nine', 'days']